In [4]:
from collections import namedtuple
import argparse
import os
import sys
import numpy as np
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset
from torch.utils.data.dataset import TensorDataset
import torch.nn.init as init
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [5]:
import os
import torch
import numpy as np

class VocabularyIndexer(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = []

    def add_word(self, word):
        if word not in self.word2idx:
            #print (word)
            self.idx2word.append(word)
            self.word2idx[word] = len(self.idx2word) - 1
        return self.word2idx[word]

    def __len__(self):
        return len(self.idx2word)


class MomentDescription(object):
    def __init__(self, path):
        self.dictionary = VocabularyIndexer()
        #print (os.path.join(path, 'description.txt'))
        self.train = self.tokenize(os.path.join(path, 'description.txt'))

    def tokenize(self, path):
        """Tokenizes a text file."""
        assert os.path.exists(path)
        # Add words to the dictionary
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = ['<eos>'] + line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    if '.' in word:
                        word = word.replace(".", "")
                    self.dictionary.add_word(word)

        # Tokenize file content and returns the tokens
        with open(path, 'r') as f:
            ids = []
            token = 0
            for line in f:
                token = 0
                words = ['<eos>'] + line.split() + ['<eos>']
                sent = np.empty(shape=(len(words),), dtype =int)
                for word in words:
                    if '.' in word:
                        word = word.replace(".", "")
                    sent[token] = self.dictionary.word2idx[word]
                    token += 1
                ids.append(sent)
            
        return ids

In [6]:
def to_variable(tensor):
    if torch.cuda.is_available():
        tensor = tensor.cuda()
    return torch.autograd.Variable(tensor)


In [7]:
args = namedtuple('args', ['batch_size', 'wordcount', 'epochs', 'cuda', 'embedding_dim', 'hidden_dim', 'clip', 'num_layers'])(120, 7614+1, 10, True, 100, 100, 0.25, 1)

In [8]:
class LSTM(nn.Module):
    def __init__(self, args):
        super(LSTM, self).__init__()
        self.wordcount = args.wordcount
        self.embedding = nn.Embedding(num_embeddings=args.wordcount, embedding_dim=args.embedding_dim)
        self.embedding.weight.data.uniform_(-0.8, 0.8)
        self.lstm1 = nn.LSTM(input_size=args.embedding_dim, num_layers=args.num_layers, hidden_size=args.hidden_dim)
        for name, param in self.lstm1.named_parameters():
            if 'bias' in name:
                init.constant(param, 0.0)
            elif 'weight' in name:
                init.uniform(param, -0.8, 0.8)
        self.projection = nn.Linear(in_features=args.hidden_dim, out_features=args.wordcount)
        self.projection.weight = self.embedding.weight

    def forward(self, input,lengths, stochastic=False):
        #print (type(input.data()))
        embeddings = self.embedding(input)
        packed = pack_padded_sequence(embeddings, lengths)
        h_t, c_t = self.lstm1(packed)
        
        output, _ = pad_packed_sequence(h_t)
        output = self.projection(output)
        return output


In [9]:
class CrossEntropyLoss3D(nn.CrossEntropyLoss):
    def forward(self, input, target):
        return super(CrossEntropyLoss3D, self).forward(input.view(-1, input.size()[2]), target.view(-1))

In [10]:
def make_pad(X,m):
    padded_samples = []
    for s in X:
        k = m - s.shape[0]
        x = np.zeros(k)
        s = np.append(s,x)
        #print (s)
        padded_samples.append(s)
    return np.array(padded_samples, dtype = int)


def get_lengths(X):
    lens = []
    for sample in X:
        lens.append(len(sample))
    return lens
    

def get_maxlen(X):
    s = 0
    sample1 = X[0]
    for sample in X:
        if s < len(sample):
            s = len(sample)
            sample1 = sample
    #print ("longest sample",sample1)      
    return s

def torchify(a,b,c):
    a = torch.LongTensor(a).contiguous()
    a = torch.transpose(a, 0, 1)
    b = torch.LongTensor(b).contiguous()
    b = torch.transpose(b, 0, 1)
    #print ("c - label", c)
    #c = torch.IntTensor(c)
    #print ("c - label", c)
   
    return a,b,c


def my_collate(batch):
    batch = sorted(batch, key=lambda x: x.shape[0], reverse=True)
    batch_size = len(batch)
    max_len = get_maxlen(batch)
    X = []
    Y = []
    
    for x in batch:
        X.append(x[:-1])
        Y.append(x[1:])
        
    print (np.array(X).shape)
    print (np.array(Y).shape)
    print ("......")
    X = np.array(X)
    lens = np.array(get_lengths(X))
    padX = make_pad(X,max_len)
    Y = np.array(Y)
    padY = make_pad(Y, max_len)
    #print ("before torchify", padX)
    padX, padY, lens = torchify(padX, padY, lens)
    #print ("......")
    #print ("first sample",(padX[0]))
    #print ("x",X)
    #print ("y",Y)
    #print ("after torchify", padX)
    print ("before train",padX.shape)
    print ("before train",padY.shape)
    return padX, padY, lens 


        

In [11]:
class LSTMDataset(Dataset):

    def __init__(self, path):
        self.path = path 
        desc = MomentDescription(self.path)
        self.train = desc.train
        


    def __len__(self):
        return len(self.train)

    def __getitem__(self, idx):
        return self.train[idx]


In [24]:
def train_model(no_epoch, model, optimizer, criterion, train_loader):
    for epoch in range(no_epoch):  # number of epochs to perform
        running_loss = 0.0
        for i, data in enumerate(train_loader, 0):
            X, Y, lens = data
            #print("XxxXXXXXXxxxx", X.shape)
            X = to_variable(X)
            Y = to_variable(Y)
            optimizer.zero_grad()
            # forward + backward + optimize
            print (lens)
            outputs = model(X, lens)
            print ("after forward",outputs.shape)
            print ("after forward",Y.shape)
            print (Y)
            #loss = criterion(outputs, Y)
            #loss.backward()
            #torch.nn.utils.clip_grad_norm(model.parameters(), args.clip)
            optimizer.step()

            # print statistics
            #running_loss += loss.data[0]
            #if i%50 == 0:
            #    print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss))
            #running_loss = 0.0 
        model_name = 'model'+str(epoch)+'.pkl'
        torch.save(model.state_dict(), model_name)
    print('Finished Training')

In [ ]:

def main():
    no_epochs = 10
    model = LSTM(args)
    #model.cuda()
    '''
    filePath = "/home/ubuntu/hw3/notebook/"+"hw3_train9.pkl"
    model_lstm = (torch.load(filePath))
    model.load_state_dict(model_lstm)
    '''
    path = ''
    train = LSTMDataset(path)
    train_loader = DataLoader(train, batch_size=6, shuffle=True, collate_fn = my_collate)
    criterion = CrossEntropyLoss3D()
    optimizer = torch.optim.SGD(model.parameters(), lr=25)
    train_model(no_epochs, model, optimizer, criterion, train_loader)

if __name__ == '__main__':
    main()


(6,)
(6,)
......
before train torch.Size([17, 6])
before train torch.Size([17, 6])
[16 10  9  8  8  4]
after forward torch.Size([16, 6, 7615])
after forward torch.Size([17, 6])
Variable containing:
  797    25   341    25    50    60
  223    57   148    91   490   214
   24   263   560    92   152   627
  249   246    25    89  3975     0
   74   426   249    55    58     0
 1972   824    74    74   847     0
  367    37    25   308   458     0
   66    25   686     0     0     0
   23   552     0     0     0     0
   58     0     0     0     0     0
   50     0     0     0     0     0
  963     0     0     0     0     0
   23     0     0     0     0     0
   28     0     0     0     0     0
  249     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 17x6]

(6,)
(6,)
......
before train torch.Size([14, 6])
before train torch.Size([14, 6])
[13 12 11 10  8  7]
after forward torch.Size([13, 6, 7615])
after forward 

after forward torch.Size([16, 6, 7615])
after forward torch.Size([17, 6])
Variable containing:
   25   215    25    50    25     1
   72   370   376   397   883   223
 2824  2915    74   188    24  1015
   18  4135    25    58   355    66
    8     5   111   796  1277   126
   82    25  2685     1   127     0
    1  1058   816   438   363     0
  797   305   817     0     0     0
 5264     7     0     0     0     0
   58  1412     0     0     0     0
   25     0     0     0     0     0
  305     0     0     0     0     0
   74     0     0     0     0     0
   25     0     0     0     0     0
  408     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 17x6]

(6,)
(6,)
......
before train torch.Size([13, 6])
before train torch.Size([13, 6])
[12 10  6  5  5  5]
after forward torch.Size([12, 6, 7615])
after forward torch.Size([13, 6])
Variable containing:
   25     1    62    21   139  3823
 1575    21   324  3399   

after forward torch.Size([9, 6, 7615])
after forward torch.Size([10, 6])
Variable containing:
   25     1    85    25    25   341
  148    62   273   438    60   560
 1197    51    25     4  3951   110
   25    25   514   348    14     0
   85   376    81     1  1312     0
  227    74    25   662     0     0
   66    25    57     0     0     0
 1069   865     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 10x6]

(6,)
(6,)
......
before train torch.Size([17, 6])
before train torch.Size([17, 6])
[16 11  9  8  7  6]
after forward torch.Size([16, 6, 7615])
after forward torch.Size([17, 6])
Variable containing:
    1  1440    96    25    50    25
    3    67   214     7  3074   673
   73   151   402   324   305  5131
   33   374    18    17    58   198
   74   998    14    25    25    60
    1    37    24   148   687     0
  295     1    25  1689     0     0
    8    71   782     0     0     0
   19   456     0     0    

after forward torch.Size([16, 6, 7615])
after forward torch.Size([17, 6])
Variable containing:
  189   148  3434   116   401  2256
   18  1197   257  6675    14    14
  370   341  1780   522  7269   148
 5472   560   110  1412    70   817
 3857    62    81   188  3551     0
  480   199   148     0     0     0
   81   380  1197     0     0     0
   25    24     0     0     0     0
  148   249     0     0     0     0
 1197    74     0     0     0     0
   37    50     0     0     0     0
  317     0     0     0     0     0
   58     0     0     0     0     0
   25     0     0     0     0     0
  188     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 17x6]

(6,)
(6,)
......
before train torch.Size([14, 6])
before train torch.Size([14, 6])
[13  8  7  6  6  6]
after forward torch.Size([13, 6, 7615])
after forward torch.Size([14, 6])
Variable containing:
    1    67    12     1  1737    50
   36   688  4907    46   

after forward torch.Size([15, 6])
Variable containing:
    1  1448    25    91  1490    46
   46    70    50   241    89    39
   24    50     4  6808   112  2009
    1    18   152   536    11     0
  139   832    25  3587  1742     0
  190   413   552   817   193     0
   87    28    58     0     0     0
   52    58     1     0     0     0
  101   458    46     0     0     0
   24   177     0     0     0     0
  249    24     0     0     0     0
   74    60     0     0     0     0
   52     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 15x6]

(6,)
(6,)
......
before train torch.Size([25, 6])
before train torch.Size([25, 6])
[24 10  9  8  8  6]
after forward torch.Size([24, 6, 7615])
after forward torch.Size([25, 6])
Variable containing:
   25    25  1412    25    50  5082
  587  4397  2375  1079   397  5468
   44    24    70     4   188  1249
   25   139  1864    17    58   152
  321   702   305    25   796 

...


(6,)
(6,)
......
before train torch.Size([10, 6])
before train torch.Size([10, 6])
[9 8 7 6 6 6]
after forward torch.Size([9, 6, 7615])
after forward torch.Size([10, 6])
Variable containing:
   25    25     1    67    25    25
  399    57   788   688    67    92
   16     4   815    40  1017  2546
 2634   152   816    52    66    25
   18   277   817     7   145  2372
    8    58   562     0     0     0
  234    15     0     0     0     0
  641     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 10x6]

(6,)
(6,)
......
before train torch.Size([12, 6])
before train torch.Size([12, 6])
[11 10  9  8  6  4]
after forward torch.Size([11, 6, 7615])
after forward torch.Size([12, 6])
Variable containing:
  148    25    46    25   555    21
 1197    36    24    50   354  2525
  322    37   189   486   191  1088
  754    25   218     1   509     0
    1   305    41  1549   101     0
 1358   178    24    74     0     0
 

after forward torch.Size([10, 6, 7615])
after forward torch.Size([11, 6])
Variable containing:
   99     1    21    25    25    46
 1398    57    24  1539    57   353
  148  2627   355  1562   367    58
  198    25   499   112    28   463
   58    50   112     0     0     0
   60   385   685     0     0     0
   18   187     0     0     0     0
    8    43     0     0     0     0
  189    33     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 11x6]

(6,)
(6,)
......
before train torch.Size([9, 6])
before train torch.Size([9, 6])
[8 8 7 7 6 5]
after forward torch.Size([8, 6, 7615])
after forward torch.Size([9, 6])
Variable containing:
    1   139   900    25  1440    21
  127  6730   367    42   355    19
   17   536    18    32  2423  4336
   74   817  1263    33   486  6620
   25    81  3285    25   363     0
  768   148  1686  1624     0     0
 1114  1197     0     0     0     0
    0     0     0     0     0     0


after forward torch.Size([18, 6, 7615])
after forward torch.Size([19, 6])
Variable containing:
   67    57    25    36   610     1
  750    24    50    24    58    46
   17    23   445   149  2257    24
    1   353    24   367    55   167
  330    58    37    58    25   627
   40   515    25   515   633     0
   25    55   621    20     0     0
 1824   848     0     0     0     0
 1197   367     0     0     0     0
  748    28     0     0     0     0
  177    18     0     0     0     0
  272   310     0     0     0     0
    1    23     0     0     0     0
   71     0     0     0     0     0
   18     0     0     0     0     0
   43     0     0     0     0     0
   33     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 19x6]

(6,)
(6,)
......
before train torch.Size([14, 6])
before train torch.Size([14, 6])
[13  9  8  7  5  5]
after forward torch.Size([13, 6, 7615])
after forward torch.Size([14, 6])
Variable co

after forward torch.Size([13, 6, 7615])
after forward torch.Size([14, 6])
Variable containing:
    1    62  1440    25   341    46
  241   311    67   148   148   688
  370    17   100   299   560    40
   42    18     1    58    11    25
  441   100   797  1301  1003    50
   70    66  1660    37     0     0
    1   101    24    25     0     0
  379    37    66   305     0     0
    7    66   145     0     0     0
   37   102     0     0     0     0
   25     0     0     0     0     0
  378     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 14x6]

(6,)
(6,)
......
before train torch.Size([9, 6])
before train torch.Size([9, 6])
[8 8 7 6 6 6]
after forward torch.Size([8, 6, 7615])
after forward torch.Size([9, 6])
Variable containing:
   25    62   560  1575    50     1
   21   196   139    37   445  4089
  413   404  2747   659    24   310
   25    17    74   536    37     5
  432    24    25  2405    99   308


after forward torch.Size([12, 6, 7615])
after forward torch.Size([13, 6])
Variable containing:
   91   341    62    92   177    57
  991   536    24  1301    70   218
 3550  4911   717    94    60   403
    1    58   148    95   373  2123
 5846     1   627    37   148     0
  629   344   127     1   257     0
 6459   190    17  2297     0     0
 6460     0     0     0     0     0
   74     0     0     0     0     0
  215     0     0     0     0     0
 4196     0     0     0     0     0
    0     0     0     0     0     0
    0     0     0     0     0     0
[torch.LongTensor of size 13x6]

(6,)
(6,)
......
before train torch.Size([13, 6])
before train torch.Size([13, 6])
[12  8  7  7  6  4]
after forward torch.Size([12, 6, 7615])
after forward torch.Size([13, 6])
Variable containing:
   25     1    25    36    25    25
  119    21    62   993  1355   643
 4666    70    51   370  1198   702
  550     1   112    71   536     0
  480  3084    81    58  2405     0
   37    19    41   378   

In [ ]:
#for i, data in enumerate(trainloader,0):
#    print(i)